Necessary Code Imports

In [3]:
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import random
import cv2

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, UpSampling2D
from keras.models import Model
from keras.layers import MaxPooling2D, Input, Conv2D, UpSampling2D, BatchNormalization
from keras.callbacks import EarlyStopping
## print("Done importing")


2023-03-16 13:00:26.349069: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Static code variables declarations

In [1]:
save_model_name = "awb"
# Size of the training images to be set before traingin
img_size = 256

#Array declartions for train/test images along with groung truth images
train_images = []
ground_truth_images = []

Collect images  - Iterate over folder to obtain images.

In [ ]:
path=r"images/Bad"
counter_train_image = 0;
for image in os.listdir(path):
    # filteration to avoid hidden file added by MacOS
    if (image != ".DS_Store"):
        # Repeatedly iterate the folder to obtain all images
        imgages = cv2.imread(os.path.join(path,image) ,cv2.IMREAD_COLOR)
        # Resize the images for normalization
        train_images_array = cv2.resize(imgages, (img_size, img_size))
        train_images.append(train_images_array)
        print("Saved train color image :",counter_train_image)
        counter_train_image+=1


path=r"images/Good"
counter_groung_image = 0;
for image in os.listdir(path):
    # filteration to avoid hidden file added by MacOS
    if (image != ".DS_Store"):
        # Repeatedly iterate the folder to obtain all images
        imgages = cv2.imread(os.path.join(path,image) ,cv2.IMREAD_COLOR)
        # Resize the images for normalization
        ground_images_array = cv2.resize(imgages, (img_size, img_size))
        train_images.append(ground_images_array)
        print("Saved ground image :",counter_groung_image)
        counter_groung_image+=1